In [9]:
import os
import django
import sys
import glob
from dotenv import load_dotenv
from datetime import datetime
import os
import requests
import voeventparse
from tracet.parse_xml import parsed_VOEvent
from utils_xml import voevent_to_dict, modify_swift_trigid, modify_swift_dates, write_and_upload

# Load environment variables from the .env file
env_path = '../.env_api'  # Update this path to the location of your .env_api file
load_dotenv(env_path)
# Set the Django settings module environment variable
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prop_api.settings')

# # Add the project root directory to the Python path
sys.path.append('../prop_api')  # Update this with the absolute path to your project root directory

# # Initialize Django
django.setup()

In [2]:
# from  proposalsettings.models.prop_test2_neutrino.model import ProposalTest2Neutrino

# prop = ProposalTest2Neutrino()
# dict_data = prop.dict() # python dict
# json_data = prop.json() # json string


# Reading the XML file

The example xml files are located in the notebooks/data directory. Other files are located in the webapp_tracet/tracet_package/tests/test_events directory. You can copy file from the test_events directory to the note book data directory and modify the date and trigger id and use. 

Please note that the topic needs to be added when writing the xml file to the database. For example, the following topic is used for SWIFT_BAT_GRB_POS:

topic = 'gcn.classic.voevent.SWIFT_BAT_GRB_POS'

In [16]:
#file_path = '/home/batbold/Projects/adacs_project_dev/TraceT/webapp_tracet/tracet_package/tests/test_events/Antares_1438351269.xml'
file_path = 'data/SWIFT_BAT_Lightcurve_new.xml'
# file_path = 'data/20241129/event_SWIFT_ACTUAL_POINTDIR_010640.xml'

with open(file_path, 'r') as file:
    xml_content = file.read()


xml content can be converted using the following formula

In [4]:
voevent_dict = voevent_to_dict(xml_content)

In [5]:
# Example usage:
new_trigid = "1351317"  # Your new TrigID
xml_content_modified = modify_swift_trigid(xml_content, new_trigid)

new_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
new_isotime = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")

xml_content_modified = modify_swift_dates(xml_content_modified, new_date, new_isotime)

In [6]:
topic = 'gcn.classic.voevent.SWIFT_BAT_GRB_POS'

In [7]:
response = write_and_upload(xml_content_modified,topic=topic)
response.status_code

201

# LVC

In [10]:
from utils_xml import modify_lvc_dates

# Assuming you want to search in the test_events directory
test_events_path = '../../test_events/'
lvc_files = glob.glob(os.path.join(test_events_path, 'LVC*.xml'))

In [11]:
file_path = 'data/LVC_real_preliminary_new.xml'

with open(file_path, 'r') as file:
    xml_content = file.read()

In [12]:
# Example usage:

new_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
updated_xml = modify_lvc_dates(xml_content, new_date)

In [13]:
topic = 'gcn.classic.voevent.LVC_REAL_PRELIMINARY'

In [14]:
response = write_and_upload(updated_xml,topic=topic)
response.status_code


201

# Working with JSON files. 

After uploading event to the database, the django web server will send proposal decision to the api with other information including event, event_group, voevents, prop_dec, event_coord. Then using the decorator log_context, the context dictionary is written to the json file the following directory:

prop_api/data/context_logs/




In [17]:
import json
from pathlib import Path

In [21]:
context_logs_dir = Path("../prop_api/data/context_logs")

# Choose a file to load (for example, the first one)
context_files = list(context_logs_dir.glob("prop*20241204*.json"))

In [22]:
context_files

[PosixPath('../prop_api/data/context_logs/prop_atca_test_grb_worth_observing_context_20241204_043410.json'),
 PosixPath('../prop_api/data/context_logs/prop_atca_test_grb_worth_observing_context_20241204_044951.json')]

In [23]:
from typing import List, Optional
from typing import List, Optional, Dict, Any
from pydantic import BaseModel
from astropy.coordinates import SkyCoord
from propsettings_app.schemas import (
    EventSchema,
    ProposalDecisionSchema,
    EventGroupSchema,
    ProposalDecision,
    SkyCoordSchema
)

from proposalsettings.models.prop_atca_test_grb.models import ProposalAtcaTestGrb
from proposalsettings.models.proposal import ProposalSettings

class ProposalDecision(BaseModel):
    id: int
    decision: str
    decision_reason: str
    proposal: ProposalAtcaTestGrb  # please remember that this model is used for the context data not(parent ProposalSettings)
    event_group_id: EventGroupSchema
    trig_id: str
    duration: float
    ra: float
    dec: float
    alt: None
    az: None
    ra_hms: str
    dec_dms: str
    pos_error: float
    recieved_data: str
    

In [24]:
context_files[0]

PosixPath('../prop_api/data/context_logs/prop_atca_test_grb_worth_observing_context_20241204_043410.json')

In [25]:
if context_files:
    context_file = context_files[0]
    print(f"Loading context from: {context_file}")

    # Read the JSON file
    with open(context_file, 'r') as f:
        context = json.load(f)

    # Parse the JSON data into a Pydantic model
    try:
        # Parse the JSON data into a Pydantic model
        event_pyd = EventSchema(**context['event'])
        prop_dec_pyd = ProposalDecision(**context["prop_dec"])
        event_group_pyd = EventGroupSchema(**context["event_group"])
        voevents_pyd = [EventSchema(**voevent) for voevent in context['voevents']]

        if context['event_coord']:
            event_coord = SkyCoordSchema(**context.event_coord.dict()).to_skycoord()
        else:
            event_coord = None
    
        context['event_coord'] = event_coord
        context['voevents'] = voevents_pyd
        context['event_group'] = event_group_pyd
        context['prop_dec'] = prop_dec_pyd
        context['event'] = event_pyd
        
        print("\nParsed context data:")
        
    except Exception as e:
        print("Validation error:", e)
        print("Error details:", e.errors())
    
else:
    print("No context files found.")
    

Loading context from: ../prop_api/data/context_logs/prop_atca_test_grb_worth_observing_context_20241204_043410.json

Parsed context data:


Here keys of the context dictionary: 

In [26]:
context.keys()


dict_keys(['event', 'prop_dec', 'voevents', 'prop_decs_exist', 'event_group', 'event_coord', 'trigger_bool', 'debug_bool', 'pending_bool', 'observation_reason', 'proposal_worth_observing', 'send_alerts', 'reached_end', 'decision_reason_log', 'proj_source_bool'])

In [27]:
print("trigger_bool: ", context['trigger_bool'])    
print("debug_bool: ", context['debug_bool'])
print("pending_bool: ", context['pending_bool'])    
print("decision_reason_log: ", context['decision_reason_log'])

trigger_bool:  False
debug_bool:  False
pending_bool:  False
decision_reason_log:  2024-12-04 04:34:09.786834+00:00: Event ID 572006: Beginning event analysis. 



# Function Implementation

Lets reimplement long grb logic using the context data and atra telescope for prop_atca_test_grb model as example. 
Here, we are writing the logic for is_worth_observing method. The input context parameter of this method is the context dictionary which is written the decorator log_context.
its written in prop_api/data/context_logs/prop_atca_test_grb_worth_observing_context_20241129_033525.json


In [28]:
from typing import Dict, List, Optional, Tuple, Union
from proposalsettings.models.prop_atca_long_grb.model import utils_grb
from proposalsettings.models.event import Event
from proposalsettings.models.telescopesettings import (
    ATCATelescopeSettings,
    BaseTelescopeSettings,
    MWATelescopeSettings,
)

# pydantic instances
prop_dec = context['prop_dec']

dec = prop_dec.dec
decision_reason_log = context['decision_reason_log']
prop_dec=prop_dec

# remember that prop_dec is instance of pydantic model and you can treate like class 
telescope_settings = prop_dec.proposal.telescope_settings


In [29]:
def is_worth_observing(context):
    prop_dec = context["prop_dec"]
    decision_reason_log = context['decision_reason_log']
    dec = prop_dec.dec
    telescope_settings = prop_dec.proposal.telescope_settings
    
    
    context_wo = worth_observing(
        event=context['event'],
        telescope_settings=telescope_settings,
        prop_dec=prop_dec,
        dec=dec,
        decision_reason_log=decision_reason_log
        )
    
    context["trigger_bool"] = context_wo["trigger_bool"]
    context["debug_bool"] = context_wo["debug_bool"]
    context["pending_bool"] = context_wo["pending_bool"]
    context["decision_reason_log"] = context_wo["decision_reason_log"]
    
    return context
    
def worth_observing(
        event: Event,
        telescope_settings: Union[
            BaseTelescopeSettings, MWATelescopeSettings, ATCATelescopeSettings
        ],
        **kwargs,
    ) -> Dict:
    """
    Determine if a GRB event is worth observing based on various criteria.

    Args:
        event (Event): The GRB event to evaluate.
        telescope_settings (Union[BaseTelescopeSettings, MWATelescopeSettings, ATCATelescopeSettings]):
            The settings for the telescope.
        **kwargs: Additional keyword arguments.

    Returns:
        Tuple[bool, bool, bool, str]: A tuple containing:
            - trigger_bool: Whether to trigger an observation.
            - debug_bool: Whether to trigger a debug alert.
            - pending_bool: Whether to create a pending observation.
            - decision_reason_log: A log of the decision-making process.
    """

    print("DEBUG - worth_observing_grb")

    prop_dec = kwargs.get("prop_dec")

    # Initialize the context with the event and default values
    context = utils_grb.initialize_context(event, kwargs)

    # Check if the event's position uncertainty is 0.0
    context = utils_grb.check_position_error(context)

    # Check if the event's position uncertainty is greater than the maximum allowed
    context = utils_grb.check_large_position_error(telescope_settings, context)

    # Check if the event's declination is within the ATCA limits
    context = utils_grb.check_atca_declination_limits(
        telescope_settings, context
    )
    print("DEBUG - context after check_atca_declination_limits")
    # Check the events likelyhood data
    context["stop_processing"] = False
    context["likely_bool"] = False

    context = utils_grb.check_fermi_likelihood(telescope_settings, context)

    context = utils_grb.check_swift_significance(telescope_settings, context)

    context = utils_grb.check_hess_significance(telescope_settings, context)

    context = utils_grb.default_no_likelihood(context)

    # Check the duration of the event
    # since new if starts, initialize the stop_processing flag
    context["stop_processing"] = False

    context = utils_grb.check_any_event_duration(telescope_settings, context)

    context = utils_grb.check_not_any_event_duration(
        telescope_settings, context
    )

    context = utils_grb.check_duration_with_limits(telescope_settings, context)

    context["reached_end"] = True
    return context



In [30]:
context = is_worth_observing(context)

print("trigger_bool: ", context['trigger_bool'])    
print("debug_bool: ", context['debug_bool'])
print("pending_bool: ", context['pending_bool'])    
print("decision_reason_log: ", context['decision_reason_log'])



DEBUG - worth_observing_grb
DEBUG - context after check_atca_declination_limits
trigger_bool:  False
debug_bool:  True
pending_bool:  False
decision_reason_log:  2024-12-04 04:34:09.786834+00:00: Event ID 572006: Beginning event analysis. 
2024-12-04 06:17:19.631496+00:00: Event ID 572006: The event's declination (44.4885) is outside limit 1 (-90 < dec < -5) or limit 2 (5 < dec < 20). 
2024-12-04 06:17:19.697861+00:00: Event ID 572006: SWIFT rate significance (25.03) >= swift_min_rate (0.000) sigma. 



# Class Implementation



In [20]:
import datetime as dt
import logging
from datetime import datetime, timezone
from enum import Enum
from typing import Dict, List, Optional, Tuple, Union

from pydantic import BaseModel, Field

from proposalsettings.consts import DEFAULT_PRIORITY, streams_all_grbs
from proposalsettings.eventtelescope_factory import EventTelescopeFactory
from proposalsettings.telescope_factory import TelescopeFactory
from proposalsettings.telescopeprojectid_factory import TelescopeProjectIdFactory


# general utils
from proposalsettings.utils import utils_helper as utils_helper
from proposalsettings.utils.utils_log import log_event


# general utils
from proposalsettings.utils import utils_helper as utils_helper
from proposalsettings.utils.utils_log import log_event

# source and triggerchoices and event and proposal classes in models
from proposalsettings.models.constants import SourceChoices, TriggerOnChoices
from proposalsettings.models.event import Event
from proposalsettings.models.proposal import ProposalSettings
from proposalsettings.models.telescope import EventTelescope, TelescopeProjectId
from proposalsettings.models.telescopesettings import (
    ATCATelescopeSettings,
    BaseTelescopeSettings,
    MWATelescopeSettings,
)

# local utils
from proposalsettings.models.prop_atca_long_grb import utils_grb
from proposalsettings.models.prop_atca_long_grb import utils_telescope_atca as utils_atca



In [21]:
logger = logging.getLogger(__name__)


class ProposalAtcaLongGrb(ProposalSettings):
    """
    Represents the settings for ATCA Long GRB proposal.
    """

    # Class variables
    streams: List[str] = [
        "FERMI_GBM_ALERT",
        "FERMI_GBM_FIN_POS",
        "FERMI_GBM_FLT_POS",
        "FERMI_GBM_GND_POS",
        "FERMI_GBM_SUBTHRESH",
        "FERMI_GBM_TEST_POS",
        "FERMI_LAT_MONITOR",
        "FERMI_LAT_OFFLINE_POS",
        "FERMI_LAT_TEST_POS",
        "FERMI_POINT_DIR",
        "HESS_GRB_TO",
        "MAXI_KNOWN_SOURCE_POS",
        "SWIFT_BAT_GRB_POS",
        "SWIFT_BAT_GRB_POS",
        "SWIFT_BAT_GRB_TEST_POS",
        "SWIFT_BAT_LIGHTCURVE",
        "SWIFT_BAT_QUICKLOOK_POS",
        "SWIFT_BAT_SCALEDMAP",
        "SWIFT_BAT_TRANS_POS",
        "SWIFT_FOM_OBS",
        "SWIFT_SC_SLEW",
        "SWIFT_UVOT_NACK_POS",
        "SWIFT_UVOT_POS",
    ]

    version: str = "1.0.0"
    id: int = 12
    proposal_id: str = "ATCA_long_GRB"
    proposal_description: str = (
        "This is the triggering proposal for the large ATCA Long GRB program"
    )
    priority: int = DEFAULT_PRIORITY
    testing: TriggerOnChoices = TriggerOnChoices.REAL_ONLY
    source_type: SourceChoices = SourceChoices.GRB

    # Initialize factories in correct order
    _telescope_factory = TelescopeFactory()
    _project_id_factory = TelescopeProjectIdFactory(
        telescope_factory=_telescope_factory
    )
    _event_telescope_factory = EventTelescopeFactory()

    # Instance variables with default values
    project_id: TelescopeProjectId = _project_id_factory.telescope_project_c3542
    event_telescope: EventTelescope = _event_telescope_factory.event_telescope_swift
    telescope_settings: ATCATelescopeSettings = ATCATelescopeSettings(
        telescope=_telescope_factory.telescope_atca,
        event_min_duration=2.056,
        event_max_duration=10000.0,
        pending_min_duration_1=0.0,
        pending_max_duration_1=2.055,
        pending_min_duration_2=0.0,
        pending_max_duration_2=0.0,
        maximum_position_uncertainty=0.1,
        observe_significant=True,
        atca_band_7mm=True,
        atca_band_7mm_freq1=33000.0,
        atca_band_7mm_freq2=35000.0,
        atca_band_15mm=True,
        atca_band_15mm_freq1=16700.0,
        atca_band_15mm_freq2=21200.0,
        atca_band_4cm=True,
        atca_band_4cm_freq1=5500.0,
        atca_band_4cm_freq2=9000.0,
        atca_band_15mm_exptime=30,
        atca_band_4cm_exptime=20,
        atca_band_7mm_exptime=30,
        atca_min_exptime=120,
    )

    class Config:
        extra = "forbid"

    def is_worth_observing(
        self, context: Dict, **kwargs
    ) -> Tuple[bool, bool, bool, str]:
        """
        Determines if an event is worth observing based on the source settings.

        Args:
            event (Event): The event to evaluate.
            **kwargs: Additional keyword arguments to pass to the worth_observing method.

        Returns:
            Tuple[bool, bool, bool, str]: A tuple containing:
                - bool: True if the event is worth observing, False otherwise.
                - bool: True if the event passes additional criteria.
                - bool: True if the event requires immediate action.
                - str: A message explaining the decision.
        """
        event = context["event"]
        prop_dec = context["prop_dec"]
        decision_reason_log = context['decision_reason_log']
        dec = prop_dec.dec

        context_wo = self.worth_observing(
            event,
            self.telescope_settings,
            prop_dec=prop_dec,
            dec=dec,
            decision_reason_log=decision_reason_log,
        )

        context["trigger_bool"] = context_wo["trigger_bool"]
        context["debug_bool"] = context_wo["debug_bool"]
        context["pending_bool"] = context_wo["pending_bool"]
        context["decision_reason_log"] = context_wo["decision_reason_log"]

        return context


    @log_event(
        log_location="end", message=f"Trigger observation completed", level="info"
    )
    def trigger_gen_observation(self, context: Dict, **kwargs) -> Tuple[str, str]:
        """
        Triggers the generation of an observation based on the event context.

        This method is called after receiving a response that an event is worth observing.
        It performs various checks and triggers observations for different telescopes (MWA, ATCA).

        Args:
            context (Dict): A dictionary containing the context of the event and observation.
            **kwargs: Additional keyword arguments.

        Returns:
            Tuple[str, str]: A tuple containing the decision and the decision reason log.
        """
        print(f"DEBUG - START context keys: {context.keys()}")

        context = utils_helper.check_mwa_horizon_and_prepare_context(context)

        # TODO: Remove this after testing
        # context["stop_processing"] = False
        print("DEBUG - context['stop_processing']: ", context["stop_processing"])

        if context["stop_processing"]:
            return context["decision"], context["decision_reason_log"]

        context = self.trigger_atca_observation(
            telescope_settings=self.telescope_settings, context=context
        )

        if (
            self.telescope_settings.telescope.name.startswith("ATCA") is False
            and self.telescope_settings.telescope.name.startswith("MWA") is False
        ):
            context["decision_reason_log"] = (
                f"{context['decision_reason_log']}{datetime.now(dt.timezone.utc)}: Event ID {context['event_id']}: Not making an MWA observation. \n"
            )

        context['reached_end'] = True
        print(f"DEBUG - END context keys: {context.keys()}")
        return context

    # GRB settings
    # event: Dict, proc_dec: Dict
    # Final aggregation function
    @log_event(
        log_location="end",
        message=f"Worth observing for GRB source completed",
        level="info",
    )
    def worth_observing(
        self,
        event: Event,
        telescope_settings: Union[
            BaseTelescopeSettings, MWATelescopeSettings, ATCATelescopeSettings
        ],
        **kwargs,
    ) -> Dict:
        """
        Determine if a GRB event is worth observing based on various criteria.

        Args:
            event (Event): The GRB event to evaluate.
            telescope_settings (Union[BaseTelescopeSettings, MWATelescopeSettings, ATCATelescopeSettings]):
                The settings for the telescope.
            **kwargs: Additional keyword arguments.

        Returns:
            Tuple[bool, bool, bool, str]: A tuple containing:
                - trigger_bool: Whether to trigger an observation.
                - debug_bool: Whether to trigger a debug alert.
                - pending_bool: Whether to create a pending observation.
                - decision_reason_log: A log of the decision-making process.
        """

        print("DEBUG - worth_observing_grb")

        prop_dec = kwargs.get("prop_dec")

        # Initialize the context with the event and default values
        context = utils_grb.initialize_context(event, kwargs)

        # Check if the event's position uncertainty is 0.0
        context = utils_grb.check_position_error(context)

        # Check if the event's position uncertainty is greater than the maximum allowed
        context = utils_grb.check_large_position_error(self.telescope_settings, context)

        # Check if the event's declination is within the ATCA limits
        context = utils_grb.check_atca_declination_limits(
            self.telescope_settings, context
        )
        print("DEBUG - context after check_atca_declination_limits")
        # Check the events likelyhood data
        context["stop_processing"] = False
        context["likely_bool"] = False

        context = utils_grb.check_fermi_likelihood(self.telescope_settings, context)

        context = utils_grb.check_swift_significance(self.telescope_settings, context)

        context = utils_grb.check_hess_significance(self.telescope_settings, context)

        context = utils_grb.default_no_likelihood(context)

        # Check the duration of the event
        # since new if starts, initialize the stop_processing flag
        context["stop_processing"] = False

        context = utils_grb.check_any_event_duration(self.telescope_settings, context)

        context = utils_grb.check_not_any_event_duration(
            self.telescope_settings, context
        )

        context = utils_grb.check_duration_with_limits(self.telescope_settings, context)

        context["reached_end"] = True
        return context

    @log_event(
        log_location="end",
        message=f"Trigger ATCA observation for GRB source completed",
        level="info",
    )
    def trigger_atca_observation(
        self,
        context: Dict,
        telescope_settings: Union[
            BaseTelescopeSettings, MWATelescopeSettings, ATCATelescopeSettings
        ],
        **kwargs,
    ) -> Tuple[str, str]:
        """
        Trigger an ATCA observation for a GRB event.

        Args:
            context (Dict): The context containing event and observation information.
            telescope_settings (Union[BaseTelescopeSettings, MWATelescopeSettings, ATCATelescopeSettings]):
                The settings for the ATCA telescope.
            **kwargs: Additional keyword arguments.

        Returns:
            Tuple[str, str]: A tuple containing updated context information.
        """

        telescope_name = telescope_settings.telescope.name

        if context["stop_processing"]:
            return context

        if telescope_name.startswith("ATCA") is False:
            return context

        print("DEBUG - Trigger ATCA observation for GRB source")

        # context = utils_tel.handle_atca_observation(context)

        context = utils_atca.handle_atca_observation(
            telescope_settings=telescope_settings, context=context
        )

        context["reached_end"] = True

        return context


It needs to be tested with the context data. 

In [22]:
prop = ProposalAtcaLongGrb()
context = prop.is_worth_observing(context)

DEBUG - worth_observing_grb
DEBUG - context after check_atca_declination_limits


In [23]:
print("trigger_bool: ", context['trigger_bool'])    
print("debug_bool: ", context['debug_bool'])
print("pending_bool: ", context['pending_bool'])    
print("decision_reason_log: ", context['decision_reason_log'])

trigger_bool:  False
debug_bool:  True
pending_bool:  False
decision_reason_log:  2024-11-29 06:06:09.619838+00:00: Event ID 572017: Beginning event analysis. 
2024-11-29 06:36:17.210795+00:00: Event ID 572017: The event's declination (44.4885) is outside limit 1 (-90 < dec < -5) or limit 2 (5 < dec < 20). 
2024-11-29 06:36:17.298909+00:00: Event ID 572017: SWIFT rate significance (25.03) >= swift_min_rate (0.000) sigma. 
2024-11-29 06:36:21.609001+00:00: Event ID 572017: The event's declination (44.4885) is outside limit 1 (-90 < dec < -5) or limit 2 (5 < dec < 20). 
2024-11-29 06:36:21.687612+00:00: Event ID 572017: SWIFT rate significance (25.03) >= swift_min_rate (0.000) sigma. 

